In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gym
import wandb
import numpy as np
from tqdm import tqdm

class Policy(nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(Policy, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)

class Value(nn.Module):
    def __init__(self, observation_space):
        super(Value, self).__init__()
        self.input_layer = nn.Linear(observation_space, 128)
        self.output_layer = nn.Linear(128, 1)

    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        state_value = self.output_layer(x)
        return state_value

class REINFORCE:
    def __init__(self, state_dim, hidden_dim, action_dim, learning_rate, gamma,
                 device, optimizer_type, v_lr):
        self.policy_net = Policy(state_dim, hidden_dim, action_dim).to(device)
        self.value_net = Value(state_dim).to(device)
        self.gamma = gamma
        self.device = device
        self.v_lr = v_lr

        if optimizer_type == 'adam':
            self.optimizer = torch.optim.Adam(self.policy_net.parameters(),
                                              lr=learning_rate)
            self.voptimizer = torch.optim.Adam(self.value_net.parameters(),
                                               lr=self.v_lr)
        elif optimizer_type == 'sgd':
            self.optimizer = torch.optim.SGD(self.policy_net.parameters(),
                                             lr=learning_rate)
            self.voptimizer = torch.optim.SGD(self.value_net.parameters(),
                                              lr=self.v_lr)
        else:
            raise ValueError("Unsupported optimizer type")

    def take_action(self, state):
        state = torch.tensor(state.reshape(1, -1), dtype=torch.float).to(self.device) 
        probs = self.policy_net(state)
        action_dist = torch.distributions.Categorical(probs) 
        action = action_dist.sample()
        return action.item()

    def update(self, transition_dict):
        reward_list = transition_dict['rewards']
        state_list = transition_dict['states']
        action_list = transition_dict['actions']

        self.optimizer.zero_grad()
        self.voptimizer.zero_grad()
        
        for i in range(len(reward_list)):  # Iterate over transitions
            reward = reward_list[i]
            state = torch.tensor(state_list[i].reshape(1, -1),
                                dtype=torch.float).to(self.device)
            action = torch.tensor(action_list[i]).view(-1, 1).to(self.device)
            
            # Calculate next state value
            next_state = torch.tensor(state_list[i+1].reshape(1, -1),
                                    dtype=torch.float).to(self.device) if i < len(reward_list) - 1 else None
            next_state_value = self.value_net(next_state) if next_state is not None else torch.tensor([[0]], dtype=torch.float).to(self.device)
            
            # Calculate target value for the value network (TD(0) update)
            target_value = reward + self.gamma * next_state_value
            
            # Get current state value
            state_value = self.value_net(state)

            # Value loss
            value_loss = F.mse_loss(state_value, target_value)

            # Accumulate gradients for value network
            value_loss.backward(retain_graph=True)

        # Perform optimization steps for both networks
        self.optimizer.step()
        self.voptimizer.step()

# agent params
gamma = 0.99
num_pbar = 10
num_episodes = 300
num_seeds = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env_name = "Acrobot-v1"

# Define the sweep configuration
sweep_config = {
    "method": "bayes",
    "metric": {"name": "regret", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"min": 1e-5, "max": 1e-3},
        "v_lr": {"min": 1e-4, "max": 1e-2},
        "hidden_dim": {"values": [32,64, 128, 256]},
        "optimizer_type": {"values": ["adam", "sgd"]}
    },
    "project": "acrobot_with_baseline",
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 3,
        "max_iter": 100
    }
}
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config)

def train(learning_rate, v_lr, hidden_dim, optimizer_type):
    env = gym.make(env_name)
    env.reset(seed=0)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    agent = REINFORCE(state_dim, hidden_dim, action_dim, learning_rate, gamma,
                      device, optimizer_type, v_lr)
    return_list = []
    for i_episode in range(num_episodes):
        episode_return = 0
        transition_dict = {
            "states": [],
            "actions": [],
            "next_states": [],
            "rewards": [],
            "dones": []
        }
        state, _ = env.reset()
        terminated, truncated = False, False
        while not terminated and not truncated:
            action = agent.take_action(state)
            next_state, reward, terminated, truncated, _ = env.step(action)
            transition_dict["states"].append(state)
            transition_dict["actions"].append(action)
            transition_dict["next_states"].append(next_state)
            transition_dict["rewards"].append(reward)
            transition_dict["dones"].append(terminated)
            state = next_state
            episode_return += reward
        return_list.append(episode_return)
        agent.update(transition_dict)

    avg = []
    for i in range(num_episodes):
        if i < 100:
            avg_return = np.mean(return_list[:i+1])
        else:
            avg_return = np.mean(return_list[i-100:i+1])
        wandb.log({"avg_scores": avg_return})
        avg.append(avg_return)

    regret = np.sum(avg)

    return -regret

def run_training():
    config_defaults = {
        "learning_rate": 1e-3,
        "v_lr": 0.005,
        "hidden_dim": 128,
        "optimizer_type": "adam"
    }
    config = wandb.init(config=config_defaults)
    learning_rate = config.config["learning_rate"]
    v_lr = config.config['v_lr']
    hidden_dim = config.config["hidden_dim"]
    optimizer_type = config.config["optimizer_type"]
    regret = train(learning_rate, v_lr, hidden_dim, optimizer_type)
    wandb.log({"regret": regret})

# Run the sweep
wandb.agent(sweep_id, function=run_training)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: jxrppr7j
Sweep URL: https://wandb.ai/rl_shobhith/acrobot_with_baseline_td0/sweeps/jxrppr7j


wandb: Agent Starting Run: q8w92x6d with config:
wandb: 	hidden_dim: 128
wandb: 	learning_rate: 0.0007002305629469535
wandb: 	optimizer_type: sgd
wandb: 	v_lr: 0.008147603237404374
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shobhith-v (rl_shobhith). Use `wandb login --relogin` to force relogin


C:\Users\Shobhith Vadlamudi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


avg_scores,▇▁▆▆█▅▃▅▅▆▅▅▅▅▄▄▅▅▆▇▇▅▆▇▇▇███▇▆▇▇▇▇▇▆▆▆▆
regret,▁
avg_scores,-378.24752
regret,113383.97091


...

wandb: Agent Starting Run: 948db8i6 with config:
wandb: 	hidden_dim: 32
wandb: 	learning_rate: 0.0005728554444161877
wandb: 	optimizer_type: sgd
wandb: 	v_lr: 0.0064864815532692105
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


avg_scores,▁▃▄▄▅▆▆▅▆▇▇▇▇▇▇████████▇▆▆▆▇▆▆▅▅▅▆▆▆▆▆▆▆
regret,▁
avg_scores,-470.19802
regret,140405.54476


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 7w2xyjur with config:
wandb: 	hidden_dim: 64
wandb: 	learning_rate: 0.0004756093031086585
wandb: 	optimizer_type: sgd
wandb: 	v_lr: 0.007465189325806406
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


avg_scores,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
regret,▁
avg_scores,-500.0
regret,150000.0


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: rbcgypxm with config:
wandb: 	hidden_dim: 64
wandb: 	learning_rate: 0.0001318502158022645
wandb: 	optimizer_type: sgd
wandb: 	v_lr: 0.004904228283436049
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
